In [15]:
import sqlite3
import pandas as pd
import numpy as np
import tarfile
import pickle
import os
import tqdm
from IPython.display import Image, display
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the data

We need to remove duplicates and NA, look for outliers.

In [16]:
SQLITE_PATH = "../raw_data/bam.sqlite"
con = sqlite3.Connection(SQLITE_PATH)
cur = con.cursor()

In [20]:
cnx = sqlite3.connect('../raw_data/bam.sqlite')

df = pd.read_sql_query("""
SELECT * FROM modules, crowd_labels where modules.mid = crowd_labels.mid
"""
, cnx)

In [21]:
df['attribute'].unique().tolist()

['content_building',
 'emotion_happy',
 'content_flower',
 'content_bicycle',
 'media_comic',
 'content_people',
 'media_3d_graphics',
 'content_dog',
 'media_vectorart',
 'emotion_scary',
 'emotion_gloomy',
 'media_graphite',
 'emotion_peaceful',
 'media_pen_ink',
 'content_cars',
 'media_oilpaint',
 'content_cat',
 'content_tree',
 'content_bird',
 'media_watercolor']

In [15]:
df['src'][0]

'https://mir-s3-cdn-cf.behance.net/project_modules/disp/e264d42588676.5601792024f48.jpg'

In [3]:
cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'").fetchall()

[('crowd_labels',),
 ('crowd_raw_hits',),
 ('crowd_raw_choices',),
 ('crowd_raw_captions',),
 ('modules',),
 ('scores',),
 ('projects',)]

In [7]:
query = """SELECT * FROM modules, crowd_labels
    limit 10;"""

In [8]:
cur.execute(query).fetchall()

[(489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  2588676,
  'content_building',
  'positive'),
 (489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  146997253,
  'content_building',
  'positive'),
 (489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  15663110,
  'content_building',
  'positive'),
 (489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  15663114,
  'content_building',
  'positive'),
 (489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  2588684,
  'content_building',
  'positive'),
 (489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  127860749,
  'co

In [18]:
query = """SELECT * FROM modules, crowd_labels WHERE modules.mid = crowd_labels.mid
    and attribute = "content_cat"
    and label="positive"
    limit 100;"""

In [19]:
cur.execute(query).fetchall()

[(160268289,
  24241935,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/6a284624241935.5633199b17b08.JPG',
  0,
  'cc by-nc',
  160268289,
  'content_cat',
  'positive'),
 (4227075,
  722050,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/dc83c34227075.5601da0e6f05e.jpg',
  0,
  'cc by-nc',
  4227075,
  'content_cat',
  'positive'),
 (104562695,
  15226981,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/8b170e15226981.5628e51323103.jpg',
  0,
  'cc by-nc',
  104562695,
  'content_cat',
  'positive'),
 (104562697,
  15226981,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/cf91fb15226981.5628e7295cbd7.JPG',
  0,
  'cc by-nc',
  104562697,
  'content_cat',
  'positive'),
 (161644557,
  24467711,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/1864cc24467711.56334b23a70ba.png',
  0,
  'cc by-nc',
  161644557,
  'content_cat',
  'positive'),
 (161644559,
  24467711,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/22619724467711

In [17]:
df = pd.read_sql("select * from scores, modules, crowd_labels
                 ",
                 sqlite3.connect("../raw_data/bam.sqlite"),
                 index_col="mid")

SyntaxError: EOL while scanning string literal (3163934022.py, line 1)

In [4]:
df

,content_bicycle,content_cat,content_tree,emotion_scary,media_oilpaint,content_bird,content_dog,emotion_gloomy,media_3d_graphics,media_pen_ink,content_building,content_flower,emotion_happy,media_comic,media_vectorart,content_cars,content_people,emotion_peaceful,media_graphite,media_watercolor
mid,,,,,,,,,,,,,,,,,,,,
489,-8.194265,-13.083484,-0.140162,-10.125443,-8.102950,-6.825498,-5.438437,-5.251916,-13.048631,-10.792016,-1.033735,-6.437519,-0.784109,-13.372336,-10.279009,-4.741513,-0.372203,-0.295248,-11.520514,-11.912029
1004,-5.908639,-9.379574,-4.740625,-4.781479,-9.769642,-7.481776,-5.466529,-1.939304,-10.137968,-9.929028,-2.769971,-1.974876,-5.214772,-9.871041,-9.538937,-8.715858,-0.110869,-3.770542,-8.218750,-17.242306
1053,-8.682774,-23.007246,-9.073513,-3.692783,-6.789311,-11.664145,-9.459610,-1.498988,-8.926823,-8.929226,-2.258289,-15.922831,-4.624222,-7.796127,-8.575558,-5.678706,-0.052687,-3.624131,-9.731888,-18.276842
1065,-7.429687,-9.030757,-3.102335,-3.109094,-7.122383,-5.884630,-6.339474,-0.523028,-6.108827,-8.709348,-0.526354,-9.055132,-5.766580,-9.137621,-8.250728,-3.525297,-1.939328,-3.715596,-7.905422,-16.156538
1067,-6.541219,-9.875690,-6.983145,-4.812078,-8.474461,-6.800467,-4.610362,-3.240254,-9.403267,-7.462641,-2.133614,-10.881371,-1.745344,-8.060266,-8.906570,-4.348733,-0.225947,-3.466125,-7.940952,-14.413774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221878331,-9.829107,-5.983519,-14.511374,-5.853211,-7.006447,-7.440907,-5.678993,-4.938928,-10.855898,-0.674897,-2.773624,-6.983993,-7.598807,-0.007607,-4.029163,-7.224268,-0.015624,-3.811826,-2.021909,-6.928348
222307913,-14.534477,-11.689569,-6.933946,-2.598703,-5.963618,-5.013757,-4.761606,-4.010435,-7.023434,-2.877161,-4.846104,-7.310925,-5.520038,-5.075095,-6.393345,-6.449655,-1.137608,-1.512154,-0.046243,-3.072185
222352655,-6.221887,-3.486142,-9.127378,-6.127351,-5.318935,-4.290847,-1.933522,-9.831583,-10.283779,-5.225632,-3.705104,-5.194139,-1.155669,-0.021087,-2.921826,-7.789520,-0.610326,-4.840499,-8.001015,-5.090885


In [5]:
res = cur.execute("""SELECT *
    FROM modules, crowd_labels
    LIMIT 100
    """)
res.fetchall()

[(489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  2588676,
  'content_building',
  'positive'),
 (489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  146997253,
  'content_building',
  'positive'),
 (489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  15663110,
  'content_building',
  'positive'),
 (489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  15663114,
  'content_building',
  'positive'),
 (489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  2588684,
  'content_building',
  'positive'),
 (489,
  40030,
  'https://mir-s3-cdn-cf.behance.net/project_modules/disp/f05608489.55f726553df57.jpg',
  0,
  'cc by-nc',
  127860749,
  'co

In [ ]:
df_yourpaintings = pd.read_csv("https://www.robots.ox.ac.uk/~vgg/data/paintings/painting_dataset_2021.csv")

# Explore the data

In [3]:
len(df_yourpaintings)

8629

In [4]:
df_yourpaintings.describe()

,Image URL,Web page URL,Subset,Labels
count,8453,8608,8629,8629
unique,8261,8416,3,89
top,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/richard-je...,'test',' boat'
freq,193,193,4301,1962


# Clean up data by dropping duplicates and missing data

In [5]:
df_yourpaintings.duplicated().sum()

156

In [6]:
df_yourpaintings = df_yourpaintings.drop_duplicates()
df_yourpaintings.describe()

,Image URL,Web page URL,Subset,Labels
count,8302,8457,8473,8473
unique,8261,8416,3,89
top,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/richard-je...,'test',' boat'
freq,42,42,4224,1921


In [7]:
len(df_yourpaintings)

8473

In [8]:
(df_yourpaintings.isnull().sum()/len(df_yourpaintings))*100

Image URL       2.018175
Web page URL    0.188835
Subset          0.000000
Labels          0.000000
dtype: float64

In [9]:
df_yourpaintings = df_yourpaintings.dropna()

In [10]:
len(df_yourpaintings)

8302

In [11]:
df_yourpaintings.describe()

,Image URL,Web page URL,Subset,Labels
count,8302,8302,8302,8302
unique,8261,8261,3,89
top,https://d3d00swyhr67nd.cloudfront.net/w1200h12...,https://artuk.org/discover/artworks/richard-je...,'test',' boat'
freq,42,42,4132,1871


For rows with working links, save image to desired path and save `filename` in a column. Then drop rows with inaccessible URLS/images.

In [ ]:
for index, row in df_yourpaintings.iterrows():
    url = row["Image URL"]
    req = Request(url)
    try:
        response = urlopen(req)
    except:
        df_yourpaintings.drop(labels=index, inplace=True, axis=0)

In [ ]:
df_yourpaintings.describe()

# Build the preprocessing pipe

First load the image data.

In [ ]:
def get_data():
    """method to get the training data (or a portion of it) from google cloud bucket"""
    df = pd.read_csv(f"gs://{BUCKET_NAME}/{BUCKET_TRAIN_DATA_PATH}", nrows=1000)
    return df

In [13]:
preprocessing_pipe = 

SyntaxError: invalid syntax (3226722706.py, line 1)

For images, apply the following standard preprocessing:
* Resize
* Crop
* Rotate
* Compression
* Grayscale: Optional

Other steps that may apply:
* Perform data augmentation
* Normalise data by / 255

# Scaling functions

# Sampling

# Encoding